In [23]:
%config Completer.use_jedi = False

In [7]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import time

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 18s 0us/step


In [5]:
local_zip = '../datasets/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('../datasets/tmp')
zip_ref.close()

base_dir = '../datasets/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [8]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [9]:
# Training and validation datasets
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [20]:
# Implementacion del modelo
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax') #despues probar con activation='sigmoid'
])

In [21]:
# Training and results
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization_15 (Batc (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 26, 26, 224)       537824    
_________________________________________________________________
batch_normalization_16 (Batc (None, 26, 26, 224)       896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 224)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 12, 12, 384)      

In [22]:
model.fit(train_generator,
          epochs=50,
          validation_data=validation_generator,
          validation_freq=1)

Epoch 1/50
100/100 [==============================] - 59s 572ms/step - loss: 1.6255 - accuracy: 0.5454 - val_loss: 0.7012 - val_accuracy: 0.5030
Epoch 2/50
100/100 [==============================] - 64s 637ms/step - loss: 1.3958 - accuracy: 0.5011 - val_loss: 0.7182 - val_accuracy: 0.5150
Epoch 3/50
100/100 [==============================] - 65s 650ms/step - loss: 1.1482 - accuracy: 0.5550 - val_loss: 0.6749 - val_accuracy: 0.5810
Epoch 4/50
100/100 [==============================] - 63s 621ms/step - loss: 1.0630 - accuracy: 0.5477 - val_loss: 0.6581 - val_accuracy: 0.6050
Epoch 5/50
100/100 [==============================] - 63s 624ms/step - loss: 0.9791 - accuracy: 0.5660 - val_loss: 0.6615 - val_accuracy: 0.6000
Epoch 6/50
100/100 [==============================] - 64s 633ms/step - loss: 0.9760 - accuracy: 0.5622 - val_loss: 0.6717 - val_accuracy: 0.6280
Epoch 7/50
100/100 [==============================] - 62s 615ms/step - loss: 0.9043 - accuracy: 0.5607 - val_loss: 0.9866 - val_ac

In [ ]:
model.evaluate(test_images)